In [1]:
import json

def load_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def find_depth(data):
    if isinstance(data, dict):
        return 1 + max((find_depth(value) for value in data.values()), default=0)
    elif isinstance(data, list):
        return max((find_depth(item) for item in data), default=0)
    return 0

# Path to the JSON file
file_path = r'/mnt/sda1/projects/Nam_exp/RAG_for_book/extracted_clauses.json'

data_dict = load_json_file(file_path)
# Calculate the depth of the data
depth = find_depth(data_dict)
print("Depth of the data structure:", depth)

Depth of the data structure: 1


In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import sys
__import__('pysqlite3')
import pysqlite3
sys.modules['sqlite3'] = sys.modules["pysqlite3"]
from langchain_chroma import Chroma

class DocumentDatabase:
    def __init__(self, persist_directory="./chroma_db"):
        self.persist_directory = persist_directory
        self.embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
        self.db_path = os.path.join(self.persist_directory, "chroma.db")
        self.pages = []
        
        # Initialize the database
        self.db = Chroma(persist_directory=self.persist_directory, embedding_function=self.embedding_function)
    def add_document(self,Document):
        self.db.add_documents(Document)
    def load_database(self):
        self.db = Chroma(persist_directory=self.persist_directory , embedding_function=self.embedding_function)
    def similarity_search(self, query, k=10):
        return self.db.similarity_search(query,k)
    
    def return_db(self):
        return self.db


In [3]:
from langchain_core.documents.base import Document
document_objects = []
for entry in data_dict:
    for clause in entry['clauses']:
        metadata = {
            'section': entry['section'],
            'title': entry['title'].strip(),
            'page_num': entry['page_num']
        }
        document = Document(page_content=entry['title'] +' '+ clause, metadata=metadata)
        document_objects.append(document)

In [4]:
db = DocumentDatabase(persist_directory="./chroma_db_claude")

/mnt/sda1/projects/Nam_exp/RAG_for_book/newlife/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/mnt/sda1/projects/Nam_exp/RAG_for_book/newlife/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
db.add_document(document_objects)

In [26]:
db.db

In [11]:
import os
from dotenv import load_dotenv
import textwrap
import google.generativeai as genai


# Load environment variables
load_dotenv()

# Configure Google Gemini API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
def make_prompt(query):
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query)

  return prompt



In [ ]:
context = docs[0].page_content

prompt = make_prompt(query, context)
print(prompt)

In [9]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')
# answer = model.generate_content(prompt)
# answer

NameError: name 'prompt' is not defined

In [12]:
from langchain_community.retrievers import BM25Retriever

In [20]:
BM25Retriever.from_documents(document_objects)

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7ffa1dfff7c0>)

In [16]:
BM25Retriever._get_value()

TypeError: get_graph() missing 1 required positional argument: 'self'

In [21]:
db.db.get()['documents']

[' Central Vacuum Systems (1)  A central vacuum cleaning system serving more than one suite or storey in a building equipped with a fire alarm system shall be designed to shut down upon activation of the fire alarm system.',
 ' Hips and Ridges (2)  Shingles referred to in Sentence (1) shall be cemented to the roof shingles and to each other with a coat of cement and fastened with nails or staples located 40 mm above the butt of the overlying shingle and 50 mm from each edge.',
 ' Slope (1)  Except as provided in Sentences (2) and (3), the slopes on which roof coverings may be applied shall conform to Table',
 ' Visibility of Exits (1)  Exits shall be located so as to be clearly visible or their locations shall be clearly indicated.',
 ' Support of Footings (2)  Granular fill shall not contain pyritic material in a concentration that would adversely affect its stability or the performance of assemblies separating dissimilar environments.',
 ' Cavity Walls (3)  The minimum thickness of c

In [1]:
import os
from dotenv import load_dotenv

from google.generativeai import configure, GenerativeModel
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from sentence_transformers import CrossEncoder
from util import make_prompt
from database import DocumentDatabase
from langchain_community.document_loaders import PyPDFLoader

# Load environment variables
load_dotenv()

# Configure Google Gemini API
configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Use @st.cache_resource for loading resources like models
def load_model():
    return CrossEncoder("mixedbread-ai/mxbai-rerank-base-v1")

# Use @st.cache_data for data processes like loading documents

def load_database():
    database = DocumentDatabase(persist_directory="./chroma_db_claude")
    return database.return_db()

def load_documents(pdf_path):
    loader = PyPDFLoader(pdf_path)
    return loader.load_and_split()

db = load_database()
model = load_model()

import json

def load_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

retriever_Chroma = db.as_retriever(search_kwargs={"k": 100})
file_path = r'/mnt/sda1/projects/Nam_exp/RAG_for_book/extracted_clauses.json'

data_dict = load_json_file(file_path)
from langchain_core.documents.base import Document
document_objects = []
for entry in data_dict:
    for clause in entry['clauses']:
        metadata = {
            'section': entry['section'],
            'title': entry['title'].strip(),
            'page_num': entry['page_num']
        }
        document = Document(page_content=entry['title'] + ' ' + clause, metadata=metadata)
        document_objects.append(document)

bm25_retriever = BM25Retriever.from_documents(document_objects)



/mnt/sda1/projects/Nam_exp/RAG_for_book/newlife/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/sda1/projects/Nam_exp/RAG_for_book/newlife/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
# Define the function
def get_answer_and_sections(query, use_reranking=True, weight_bm25=0.2):
    weights = [weight_bm25, 1.0 - weight_bm25]
    ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever_Chroma], weights=weights)
    
    # Document retrieval
    docs = ensemble_retriever.get_relevant_documents(query)

    if docs:
        if use_reranking:
            # Rerank the retrieved documents
            documents = [doc.page_content for doc in docs]
            ranked_indices = model.rank(query, documents, return_documents=False, top_k=3)
            indices = [ranked_indice['corpus_id'] for ranked_indice in ranked_indices]
            ranked_docs = [docs[idx] for idx in indices]
        else:
            # Use the top 3 documents without reranking
            ranked_docs = docs[:3]

        combined_text = " ".join([doc.page_content for doc in ranked_docs])

        prompt = make_prompt(query, combined_text)
        gen_model = GenerativeModel('gemini-1.0-pro-latest')
        answer = gen_model.generate_content(prompt)

        # Extract metadata sections
        sections = [doc.metadata['section'] for doc in ranked_docs]
        print(answer)

        return answer.candidates[0].content.parts[0], sections
    else:
        return "No relevant documents found.", []

In [5]:
import pandas as pd
file_path = "/mnt/sda1/projects/Nam_exp/RAG_for_book/test_per.xlsx"
df = pd.read_excel(file_path)

In [6]:
df

,QUESTION,ANSWER,REFERENCE
0,What is the spacing requirement for weep holes...,NaN,NaN
1,What is the minimum allowable length of bearin...,NaN,NaN
2,What are the regulations for holes that may be...,NaN,NaN
3,What is the minimum width of an unobstructed h...,NaN,NaN
4,What is the regulation for wood columns in con...,NaN,NaN
5,What is the maximum force required to be exert...,NaN,NaN
6,What is the minimum depth for wooden stair str...,NaN,NaN
7,What are the regulations for the finish of sta...,NaN,NaN
8,What is the purpose of exits?,NaN,NaN
9,What is the minimum height at doorways?,NaN,NaN


In [26]:
row

QUESTION     Where must flashing be installed in a masonry ...
ANSWER                                                     NaN
REFERENCE                                                  NaN
Name: 29, dtype: object

In [30]:
for index, row in df.iterrows():
    if index>28:
        question = row['QUESTION']
        answer, sections = get_answer_and_sections(question)
        df.at[index, 'ANSWER'] = answer.text
        df.at[index, 'REFERENCE'] = 'Division B, '+sections[0]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "finish_reason": "RECITATION",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 352,
        "total_token_count": 352
      }
    }),
)


IndexError: list index (0) out of range

In [27]:
sections

['9.17.5.2.', '9.36.2.2.', '9.17.4.1.']

In [28]:
question

'Where must flashing be installed in a masonry veneer wall?'

In [34]:
df.to_excel("result_18_08.xlsx")